# Test Env

In [4]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import *
import random

In [5]:
adj_matrix = [[0, 1, 0, 1, 0, 0],
[0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 1, 1],
[0, 1, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 1]]

ADJ_MATRIX_TO_EDGE_SELECTOR = "{i, j: int | 1 in (list.idx[i]).idx[j]}"

# These are 1 indexed instead of 0 indexed.
directedgraph = inferredEdge(selector=ADJ_MATRIX_TO_EDGE_SELECTOR, name="")(adj_matrix)
directedgraph = hideAtom(selector="list")(directedgraph)


In [ ]:
import random

STRUCTURE = "unweighted_graph_adjmatrix"
for size in [10]:
    # Generate random adjacency matrix
    adj_matrix = [[random.randint(0, 1) if i != j else 0 for j in range(size)] for i in range(size)]
    
    directedgraph = inferredEdge(selector=ADJ_MATRIX_TO_EDGE_SELECTOR, name="")(adj_matrix)
    directedgraph = hideAtom(selector="list")(directedgraph)
    #evaluate(directedgraph, method="browser")
    diagram(directedgraph, method="browser", perf_path="test-perf.json", perf_iterations=30, headless=True, timeout=600)


In [7]:
def adj_matrix_two_components(size, split=None, extra_prob=0.0, seed=None):
    """
    Return an NxN adjacency matrix (0/1 ints) for an undirected graph
    with exactly 2 connected components.

    Args:
        size (int): total number of nodes (must be >= 2)
        split (int|None): size of first component (1 <= split <= size-1).
                          If None, uses floor(size/2).
        extra_prob (float): probability to add extra intra-component edges (0..1)
        seed (int|None): random seed

    Returns:
        list[list[int]]: NxN symmetric adjacency matrix with zeros on diagonal
    """
    import random
    if size < 2:
        raise ValueError("size must be >= 2")
    if seed is not None:
        random.seed(seed)

    if split is None:
        split = size // 2
    if not (1 <= split <= size - 1):
        raise ValueError("split must be between 1 and size-1")

    n = size
    mat = [[0] * n for _ in range(n)]

    def make_connected_block(start, block_size):
        # create a simple chain to ensure connectivity
        for i in range(start, start + block_size - 1):
            a, b = i, i + 1
            mat[a][b] = mat[b][a] = 1
        # optional extra random edges inside the block
        for i in range(start, start + block_size):
            for j in range(i + 2, start + block_size):
                if random.random() < extra_prob:
                    mat[i][j] = mat[j][i] = 1

    make_connected_block(0, split)
    make_connected_block(split, n - split)

    return mat

# Example usage:
# adj = 
# for row in adj: print(row)




STRUCTURE = "scc_graph"
for size in [30]:
    # Generate adjacency matrix with cycles for SCCs
    adj_matrix = [adj_matrix_two_components(size, split=3, extra_prob=0.1, seed=42)]
    
    two_comp = inferredEdge(selector=ADJ_MATRIX_TO_EDGE_SELECTOR, name="")(adj_matrix)
    two_comp = hideAtom(selector="list")(two_comp)
    
    sss = f"(int->int) & {{u, v : int | ((u->v +v->u) in ^{ADJ_MATRIX_TO_EDGE_SELECTOR}) }}"
    two_comp = group(selector=sss, name="SCC")(two_comp)
    diagram(two_comp, method="browser")

